# Introduction to ipyautoui

ipyautoui is a Python package that provides a simple way to create interactive user interfaces in Jupyter notebooks. It is built on top of Jupyter widgets and Pydantic and provides a higher-level API for creating interactive widgets.  It is particularly well suited for creating settings panels.


## TO DO IN THIS NOTEBOOK
- [ ] Add a brief description of the package and its history?
- [ ] Show how to use ipyautoui to create a simple settings panel relatively quickly
  - [ ] Mention the active development of the package and the fact that it is still a bit buggy
  - [ ] Mention other features of Pydantic that may be relevant to our dashboard
- [ ] Have them replace existing settings widgets with control widget generated with ipyautoui
- [ ] Connect new control widget to dashboard

# APPARENTLY WE CANT EXPORT FROM TWO NOTEBOOKS

In [ ]:
#| default_exp widgets_pydantic

## Making a widget with ipyautoui takes a couple of steps

1. Define a pydantic model (or write a jason schema)
2. Feed the model to AutoUi
3. You get back a widget! With a value! Whose value is easy to save!

## An example

### Exercise

In the cell below, create a pydantic model called `SimpleModel` with one field, called `window_size`, that is an integer.

In [ ]:
#TODO move to answers 
from pydantic import BaseModel

class SimpleModel(BaseModel):
    window_size : int

To make a widget from this, run the cell below. `AutoUi` takes in the pydantic model and turns it into a widget. Note that it is the class itself, not an instance of the class, that is the argument to `AutoUi`.

In [ ]:
from ipyautoui import AutoUi

ui = AutoUi(SimpleModel)

Run the cell below to display the widget and try changing its value. Note that you can only type numbers (with out decimals) into the box. 

In [ ]:
ui

### Attributes and methods of an AutoUi generated widget

A couple of attributes of the auto-generated widget are useful to know about. 

1. `di_widgets` -- dictionary of widgets, one widget for each pydantic field.
2. `value` -- dictionary of values, one key and value for each pydantic field.

Though one could use `di_widgets` to display the widget for a field outside the auto-generated widget, that is rarely useful. It is sometimes handy, though, to explore the properties of that widget.

In [ ]:
ui.di_widgets

In this case, `AutoUi` is representing the `int` in our model with an `IntText` widget.

The `value` for an `AutoUi`-generated widget is a dictionary. The keys are the fields defined in the pydantic model and the values are the current value of that field.

In [ ]:
ui.value

You can set the value, like with any other widget:

In [ ]:
ui.value = {"window_size": 11}

The value can also be observed, though there is a twist -- it is `_value` that you observe, not `value. The observer below simply prints the value of the widget.


In [ ]:
def print_value(change):
    print(f"{change['new']=}")

In [ ]:
# NOTE that we are observing changes in _value rather than value
ui.observe(print_value, "_value")

### Exercise

Change the value of `ui` in the cell below and confirm that you get the message you expect.

In [ ]:
ui.value = {"window_size": 81}

In [ ]:
ui.value = {"window_size": 84}

Add contraint to window size, one constraint at a time, to see how they affect the widget that is generated.

In [ ]:
from typing import Annotated
from pydantic import Field

class SimpleModel2(BaseModel):
    window_size : Annotated[int, Field(gt=1)]

In [ ]:
ui2 = AutoUi(SimpleModel2)
ui2

Oh my! Apparently the widget is sort of ok with values that are not consistent with the pydantic model.

In [ ]:
ui2.di_widgets

In [ ]:
class SimpleModel3(BaseModel):
    window_size : Annotated[int, Field(ge=2, json_schema_extra=dict(autoui='ipywidgets.BoundedIntText'))]

In [ ]:
ui3 = AutoUi(SimpleModel3)
ui3

In [ ]:
ui3.di_widgets

## NEED MORE INTRO BEFORE WE DO THIS

In [ ]:
from ipyautoui import AutoUi
from dashboard.widgets_pydantic import DataSelectorModelDraft6

In [ ]:
controls = AutoUi(DataSelectorModelDraft6)

In [ ]:
controls

In [ ]:
from pydantic import BaseModel, Field, model_validator
from typing import Annotated

In [ ]:
#| export

class DataSelectorModel(BaseModel, validate_assignment=True):
    year_range: Annotated[
        tuple[int, int],
        Field(json_schema_extra=dict(autoui='ipywidgets.IntRangeSlider'))
    ] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2, le=100, json_schema_extra=dict(autoui='ipywidgets.BoundedIntText'))] = 2
    polynomial_order: Annotated[int, Field(ge=1, le=10)] = 1

    # mode="after" means the validator runs after pydantic has checked that the individual
    # fields have values that are valid.
    @model_validator(mode="after")
    def limit_polynomial_order(self):
        
        if self.polynomial_order > self.window_size - 1:
            # Handle a bad polynomial order or window size
            raise ValueError("Polynomial order must be smaller than window size")
            
        # If we got this far the polynomial order is consistent with the window size
        # so return self. Failing to return self will end up causing an error.
        return self

In [ ]:
DataSelectorModel.model_json_schema()

In [ ]:
ui = AutoUi(DataSelectorModel)
ui

In [ ]:
ui.value

In [ ]:
ui.di_widgets

In [ ]:
from ipyautoui import demo
demo()

In [ ]:
from nbdev.export import nb_export

nb_export('03b_ipyautoui.ipynb', 'dashboard')